This notebook is for collecting and cleaning all the ability data for pokemon up to generation 5. This is then stored and exported into a csv file

# Libraries

In [2]:
# Libraries
import pokebase as pb
import requests
import json
import pandas as pd
import numpy as np

# Collect the Data

In [8]:
query = requests.get('https://pokeapi.co/api/v2/ability?limit=100000&offset=0')
query_data = json.loads(query.text)
print(query_data.keys())

dict_keys(['count', 'next', 'previous', 'results'])


In [9]:
# get the abilities
ability_data = query_data["results"]
# ability_data

[{'name': 'stench', 'url': 'https://pokeapi.co/api/v2/ability/1/'},
 {'name': 'drizzle', 'url': 'https://pokeapi.co/api/v2/ability/2/'},
 {'name': 'speed-boost', 'url': 'https://pokeapi.co/api/v2/ability/3/'},
 {'name': 'battle-armor', 'url': 'https://pokeapi.co/api/v2/ability/4/'},
 {'name': 'sturdy', 'url': 'https://pokeapi.co/api/v2/ability/5/'},
 {'name': 'damp', 'url': 'https://pokeapi.co/api/v2/ability/6/'},
 {'name': 'limber', 'url': 'https://pokeapi.co/api/v2/ability/7/'},
 {'name': 'sand-veil', 'url': 'https://pokeapi.co/api/v2/ability/8/'},
 {'name': 'static', 'url': 'https://pokeapi.co/api/v2/ability/9/'},
 {'name': 'volt-absorb', 'url': 'https://pokeapi.co/api/v2/ability/10/'},
 {'name': 'water-absorb', 'url': 'https://pokeapi.co/api/v2/ability/11/'},
 {'name': 'oblivious', 'url': 'https://pokeapi.co/api/v2/ability/12/'},
 {'name': 'cloud-nine', 'url': 'https://pokeapi.co/api/v2/ability/13/'},
 {'name': 'compound-eyes', 'url': 'https://pokeapi.co/api/v2/ability/14/'},
 {'na

In [50]:
test = ability_data[40]
# ability_link = test["url"]
print(test)

ability_query = requests.get(test["url"])
ability_query_data = json.loads(ability_query.text)
print(ability_query_data.keys())

{'name': 'water-veil', 'url': 'https://pokeapi.co/api/v2/ability/41/'}
dict_keys(['effect_changes', 'effect_entries', 'flavor_text_entries', 'generation', 'id', 'is_main_series', 'name', 'names', 'pokemon'])


In [40]:
# print(ability_query_data)

In [51]:
ability_query_data["generation"]

{'name': 'generation-iii', 'url': 'https://pokeapi.co/api/v2/generation/3/'}

In [53]:
type(ability_query_data["is_main_series"])

bool

In [28]:
ability_query_data["name"]

[{'effect_entries': [{'effect': 'Hat im Kampf keinen Effekt.',
    'language': {'name': 'de',
     'url': 'https://pokeapi.co/api/v2/language/6/'}},
   {'effect': 'Has no effect in battle.',
    'language': {'name': 'en',
     'url': 'https://pokeapi.co/api/v2/language/9/'}}],
  'version_group': {'name': 'black-white',
   'url': 'https://pokeapi.co/api/v2/version-group/11/'}}]

# Data Cleaning

In [31]:
gens = ['generation-i', 'generation-ii', 'generation-iii', 'generation-iv', 'generation-v']
latest_game = "back-2-white-2"
abilities_df = pd.DataFrame(columns=["name", "generation", "effects", "description"])

In [66]:
for ability in ability_data:
    # pull the page data that we need
    ability_query = requests.get(ability["url"])
    ability_query_data = json.loads(ability_query.text)
   
    if ability_query_data["is_main_series"]: # only get the abilities from the main series games
        # print("got here")
        if ability_query_data["generation"]["name"] in gens: # only get the abilities present inthe generations we are looking at
            # print("here")
            temp = ["", "", "", ""]
            
            temp[0] = ability_query_data["name"] # get the ability's name
            
            # get the generation that the ability is from
            gen = ability_query_data["generation"]["name"]
            
            if "iv" in gen:
                temp[1] = 4
            elif "v" in gen:
                temp[1] = 5
            elif "iii" in gen:
                temp[1] = 3
            elif "ii" in gen:
                temp[1] = 2
            elif "i" in gen:
                temp[1] = 1
            else: # raise an error if we get a value that is not one of these generations
                print(gen)
                raise ValueError("We got an invalid generation!")
                
            effect_data = ability_query_data["effect_entries"] # get the effect description
            for text in effect_data:
                if text["language"]["name"] == "en": # get the english descriptions
                        temp[2] = text["effect"]
                        break
                        
            flavor_data = ability_query_data["flavor_text_entries"]
            for text in flavor_data:
                # print(text)
                if text["language"]["name"] == "en": # get the english descriptions
                        temp[3] = text["flavor_text"]
                        break
                        
            # print(temp)
            
            abilities_df.loc[len(abilities_df.index)] = temp                

In [67]:
abilities_df

,name,generation,effects,description
0,stench,3,This Pokémon's damaging moves have a 10% chanc...,Helps repel wild POKéMON.
1,drizzle,3,The weather changes to rain when this Pokémon ...,Summons rain in battle.
2,speed-boost,3,This Pokémon's Speed rises one stage after eac...,Gradually boosts SPEED.
3,battle-armor,3,Moves cannot score critical hits against this ...,Blocks critical hits.
4,sturdy,3,"When this Pokémon is at full HP, any hit that ...",Negates 1-hit KO attacks.
...,...,...,...,...
352,iron-barbs,5,Whenever a move makes contact with this Pokémo...,Inflicts damage to the\nPokémon on contact.
353,zen-mode,5,This Pokémon switches between Standard Mode an...,Changes the Pokémon’s\nshape when HP is halved.
354,victory-star,5,"All friendly Pokémon's moves, including this P...",Boosts the accuracy of its\nallies and itself.
355,turboblaze,5,This Pokémon's moves completely ignore abiliti...,Moves can be used\nregardless of Abilities.


In [68]:
abilities_df.to_csv("ability_data.csv")